#### Cluster TCRs based on TCR Clone size

In [ ]:
# MFuzz
library(Mfuzz)
library(magrittr)

df <- read.csv("/path/UCART_SLE_PatientTcell_TracingTCRClone.csv",row.names="X")
df <- df[!df$Sample %in% c("cart"),]
df <- df[!df$dat6_origin %in% c("UCART_origin"),]
df <- df[df$AIRR_Type_TCR %in% c("TCR"),]
df$Sample <- factor(df$Sample, levels = c("preTreat","day6","day60","day91","day123","day180","day361"))
df <- df %>% dplyr::add_count(Sample, Trace_Clonotype, name = "Sample_Trace_Clonotype_Size")

df_plot <- df[,c("Sample","Sample_Trace_Clonotype_Size","Trace_Clonotype")] %>% dplyr::distinct()
df_plot$Sample <- factor(df_plot$Sample, levels = c("preTreat","day6","day60","day91","day123","day180","day361"))

df_wide <- df_plot %>%
  tidyr::pivot_wider(names_from = Sample, values_from = Sample_Trace_Clonotype_Size, values_fill = list(Value = 0))

df_wide[is.na(df_wide)] <- 0
df_wide <- as.data.frame(df_wide)
rownames(df_wide) <- df_wide$Trace_Clonotype
df_wide <- df_wide[,-1]
df_wide_drop <- df_wide[rowSums(df_wide >= 10, na.rm = TRUE) > 0,]

df_wide_drop <- as.matrix(df_wide_drop)
mfuzz_class <- new('ExpressionSet',exprs = df_wide_drop)

mfuzz_class <- standardise(mfuzz_class)

cluster_num <- 6
set.seed(2025)
mfuzz_cluster <- mfuzz(mfuzz_class, c = cluster_num, m = mestimate(mfuzz_class))

mfuzz_cluster$size
mfuzz_cluster$cluster[mfuzz_cluster$cluster == 1]
table(mfuzz_cluster$cluster)


library(RColorBrewer)
color.2 <- colorRampPalette(rev(c("#ff0000", "Yellow", "OliveDrab1")))(1000)
pdf('/path/PatientTcells_7Samples_TCR_Tracing_mfuzz_clusters_plot.pdf',height=10,width=10)
mfuzz.plot(mfuzz_class, mfuzz_cluster, mfrow=c(3,2),
    new.window=FALSE,
    time.labels=colnames(mfuzz_class),
    colo=color.2)
dev.off()

# save cluster results
mfuzz_cluster_cluster <- as.data.frame(mfuzz_cluster$cluster)
write.csv(mfuzz_cluster_cluster, file = "/path/PatientTcells_7Samples_TCR_Tracing_mfuzz_clusters_DropCloneSizeless10.csv")

In [ ]:
### Cluster 4
mfuzz_cluster_cluster <- read.csv('/path/PatientTcells_7Samples_TCR_Tracing_mfuzz_clusters_DropCloneSizeless10.csv',header = T,row.names = 1)
mfuzz_cluster_cluster$TCR_CloneID <- rownames(mfuzz_cluster_cluster)
Clu4 <- mfuzz_cluster_cluster[which(mfuzz_cluster_cluster$mfuzz_cluster.cluster == 4),]
df_wide4 <- df_wide_drop[rownames(df_wide_drop) %in% rownames(Clu4),]
range(df_wide4)

color = c(
  colorRampPalette(colors = c("white", "#F5BC88"))(length(seq(0,10,by=1))),
  colorRampPalette(colors = c("#F5BC88","#DA5315"))(length(seq(10,100,by=10))),
  colorRampPalette(colors = c("#DA5315","#7F0000"))(length(seq(100,555,by=50)))
)


library(gridExtra)

pdf("/path/SLE_PatientTcells_7Samples_TCR_Tracing_heatmap_Clu4.pdf", width = 6, height = 12)

p <- pheatmap(df_wide4,
            # scale='row',
            cluster_rows = T,
            cluster_cols = F,
            clustering_method = "single", 
            show_rownames = T,
            show_colnames = T,
            width = 6,
            height = 8, 
            color = color,
            display_numbers = TRUE,
            number_format = "%.0f",
            main = "TCR Clone Tracing (Clu4)",
            angle_col = 90)
dev.off()